## Documentation

To read more about the Time Series Data Stream, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/tsds.html).

![time_series_docs](../images/time_series_docs.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(hosts=HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'iugjHCt8SwCWRVd35xnJ0A',
 'name': '5013781c82bc',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-02-05T22:10:57.067596412Z',
             'build_flavor': 'default',
             'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.2'}}


## ILM policy

If you want to modify other phases of the policy, check this [example](https://www.elastic.co/guide/en/elasticsearch/reference/current/set-up-tsds.html#tsds-ilm-policy) in the docs.

In [5]:
from pprint import pprint

policy = {
    "phases": {
        "hot": {
            "actions": {
                "rollover": {
                    "max_age": "5m"
                }
            }
        },
        "delete": {
            "min_age": "20m",
            "actions": {
                "delete": {}
            }
        }
    }
}

ILM_NAME = "cpu_usage_policy_v2"

response = es.ilm.put_lifecycle(name=ILM_NAME, policy=policy)
pprint(response.body)

{'acknowledged': True}


## Index template

In [6]:
TEMPLATE_NAME = "cpu_example_template"


response = es.indices.put_index_template(
    name=TEMPLATE_NAME,
    index_patterns=[
        "cpu_example_template*" # Applies to any index starting with 'cpu_usage_data'
    ],
    data_stream={},
    template={
        "settings": {
            "index.mode": "time_series", # Enable time series data mode
            "index.lifecycle.name": ILM_NAME # Apply the ILM policy
        },
        "mappings": {
            "properties": {
                "@timestamp": {
                    "type": "date"
                },
                "cpu_usage": {
                    "type": "float",
                    "time_series_metric": "gauge"
                },
                "cpu_name": {
                    "type": "keyword",
                    "time_series_dimension": True
                }
            }
        }
    },
    priority=500, # A priority higher than 200 to avoid collisions with built-in templates
    meta={
        "description": "Template for CPU usage data"
    },
    allow_auto_create=True
)

pprint(response.body)

{'acknowledged': True}


## Index data

In [8]:
import psutil
from datetime import datetime, timezone

index_alias = "cpu_example_template"

while True:
    document = {
        "@timestamp": datetime.now(timezone.utc),
        "cpu_usage": psutil.cpu_percent(interval=0.1),
        "cpu_name": "17-13650HX"
    }

    es.index(index=index_alias, document=document, refresh=True)

KeyboardInterrupt: 